Import Libraries

In [1]:
import csv
import numpy as np
import tensorflow as tf
import datetime
import os
from tensorflow.keras import backend as K

In [2]:
attack_set_1_raw = np.genfromtxt('zodiac_attacked.csv', 
                             delimiter=',',
                             skip_header=2)

no_attack_set_1_raw = np.genfromtxt('zodiac_not_attacked.csv', 
                             delimiter=',',
                             skip_header=2)

In [3]:
attack_set_1 = np.take(attack_set_1_raw, [1,2,3,4,15], axis=1)
no_attack_set_1 = np.take(no_attack_set_1_raw, [1,2,3,4,15], axis=1)
training_set = np.concatenate((attack_set_1, no_attack_set_1), axis=0)

In [4]:
attack_set_1

array([[8.14870000e+04, 0.00000000e+00, 5.71780000e+04, 7.68000000e+02,
        1.00102465e+01],
       [8.14740000e+04, 1.00000000e+00, 1.05392000e+05, 2.05100000e+03,
        1.00103073e+01],
       [8.17790000e+04, 1.00000000e+00, 9.68790000e+04, 1.85600000e+03,
        1.00102484e+01],
       ...,
       [5.23820000e+04, 2.69710000e+04, 6.00320000e+04, 5.44960000e+04,
        1.00103407e+01],
       [5.21710000e+04, 2.65700000e+04, 8.84520000e+04, 2.30360000e+04,
        1.00102460e+01],
       [5.25490000e+04, 2.68910000e+04, 3.63240000e+04, 1.77220000e+04,
        1.00103118e+01]])

In [39]:
zeroes_set = np.zeros((len(no_attack_set_1),1))
ones_set = np.ones((len(attack_set_1),1))
target_set = np.concatenate((ones_set, zeroes_set), axis=0)

In [54]:
# Default Model
tf.keras.backend.clear_session()

input_size = 5
output_size = 1

model = tf.keras.Sequential([
    tf.keras.layers.Dense(
        5, 
        activation='relu', 
        input_shape=(input_size,),
        kernel_initializer='glorot_normal',
    ),
#     tf.keras.layers.Dense(
#         5, 
#         activation='relu',
#     ),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(optimizer=tf.keras.optimizers.Adam(), 
                loss='binary_crossentropy', 
                metrics=['accuracy'])

In [55]:
modelStartedTime = datetime.datetime.now().strftime("%Y_%m_%dt%H_%M_%S")

log_dir = os.path.join(
    "logs",
    "adam_optimizer",
    "loss_binary_crossentropy"
    # "one_hidden_layer",
    "two_hidden_layers",
    "validation_split_0D1",
    modelStartedTime,
)

# log_dir = os.path.join(
#     "logs",
#     "test",
#     modelStartedTime,
# )

callbacks = [
    # tf.keras.callbacks.EarlyStopping(patience=50, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        
        #update_freq='epoch'        
    )
]

In [58]:
model.fit(
    training_set, 
    target_set, 
    initial_epoch=0,
    # initial_epoch=init_epoch-epoch_count,
    epochs=20,
    validation_split=0.1, 
    # batch_size=20, 
    verbose=2, 
    shuffle=True,
    callbacks=callbacks, 
)

Train on 2338 samples, validate on 260 samples
Epoch 1/20
2338/2338 - 0s - loss: 3.5689 - accuracy: 0.8743 - val_loss: 1.9643 - val_accuracy: 0.7769
Epoch 2/20
2338/2338 - 0s - loss: 2.6707 - accuracy: 0.8725 - val_loss: 1.2349 - val_accuracy: 0.9500
Epoch 3/20
2338/2338 - 0s - loss: 2.7627 - accuracy: 0.8832 - val_loss: 1.2387 - val_accuracy: 0.9500
Epoch 4/20
2338/2338 - 0s - loss: 2.0019 - accuracy: 0.8854 - val_loss: 1.2651 - val_accuracy: 0.9500
Epoch 5/20
2338/2338 - 0s - loss: 3.2443 - accuracy: 0.8708 - val_loss: 1.3712 - val_accuracy: 0.9077
Epoch 6/20
2338/2338 - 0s - loss: 1.8031 - accuracy: 0.8973 - val_loss: 1.4380 - val_accuracy: 0.7769
Epoch 7/20
2338/2338 - 0s - loss: 1.1809 - accuracy: 0.8905 - val_loss: 1.2022 - val_accuracy: 0.9500
Epoch 8/20
2338/2338 - 0s - loss: 1.8057 - accuracy: 0.8935 - val_loss: 1.2122 - val_accuracy: 0.9500
Epoch 9/20
2338/2338 - 0s - loss: 1.8603 - accuracy: 0.8926 - val_loss: 1.2791 - val_accuracy: 0.9077
Epoch 10/20
2338/2338 - 0s - loss: 

In [59]:
model.save('port_model_2.h5')

In [9]:
model.layers[0].get_weights()

[array([[ 0.12517838, -0.03866845, -0.20502004, -0.75644237, -0.14499573],
        [-0.19907333, -0.6377549 ,  0.41524097, -0.4151665 , -0.3867644 ],
        [ 0.51228166, -0.16080315,  0.34544232,  0.06832434, -0.27374703],
        [ 0.33012253, -0.7053988 ,  0.21261647, -0.65571713,  0.9632727 ],
        [ 0.15257913,  0.05354644,  0.7596836 , -0.13905106, -0.12730189]],
       dtype=float32),
 array([-0.22902527,  0.        ,  0.3487387 ,  0.        ,  0.37249017],
       dtype=float32)]

In [19]:
tf.keras.experimental.export_saved_model(model, 'best_model_v2' + modelStartedTime + '.h5')

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO

In [26]:
model.save('port_model.h5')

In [35]:
loaded_model = tf.keras.models.load_model('best_model_2019_10_20t21_32_39.h5')

In [36]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 30        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________
